## <span style="color:grey">Initialization</span>

In [1]:
%run "KiteConnect_AlgoBase.ipynb" 

##  <span style="color:green">Trading Strategy </span>

In [2]:
class algoTrade(algoTrade_base):
   
    def __init__(s, symbol):
        logger.debug("AlgoTrade Called: "+symbol)
        super(algoTrade, s).__init__(symbol)
        
    
    # Long Strategies
    def long_indicators(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
        temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
        temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
        temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
        temp_df = temp_df | (REF(s.rsi,1) <=50)
        #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
        #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)
        
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_indicators(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
        temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
        temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
        temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
        temp_df = temp_df | (REF(s.rsi,1) >=50)
        #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
        return pd.DataFrame( temp_df, columns=["sell"])
    

    
    def long_breakout(s):
        temp_df = s.haCLOSE >= s.BBT.shift(1)
        temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_breakout(s):
        temp_df = s.haOPEN <= s.BBB.shift(1)
        temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_ha(s):
        temp_df = (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN) 
        #temp_df = temp_df & (s.rsi < 40 )
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_ha(s):
        temp_df = (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
        #temp_df = temp_df & (s.rsi > 60 )
        return pd.DataFrame( temp_df , columns=["sell"])

    def tradeDecision(s, price_ohlc_df):
        global blackoutEnabled

        super(algoTrade, s).tradeDecision(price_ohlc_df)
        
        #s.rsi = RSI(s.CLOSE, timeperiod=20)
        #s.min = MIN(s.CLOSE, timeperiod=30)
        #s.max = MAX(s.CLOSE, timeperiod=30)
        
        s.buy  = s.long_ha() & s.long_indicators()
        s.sell = s.short_ha() & s.short_indicators()
        
        s.postprocess()
        
        return (s.buy,s.sell)
        
if mode == "Algo":
    x = 'YESBANK'
    gc.collect()
    days=1
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,5,3,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, cd=True, plot=True)+"%")
    
    

   YESBANK 	 3.85 	 2.19%


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]




Portfolio return: 2.19%


In [ ]:
# Backtesting for no of days

if mode == "Algo":
    x = 'YESBANK'
    gc.collect()
    days=2
    portfolio = pd.DataFrame([x])
    
    noofdays = 180
    for i in np.linspace(noofdays,1,noofdays):
        toDate = dt.datetime(2019,5,2,15,15) - dt.timedelta(days=i)
        #print(toDate)
        if isholiday(toDate) == True:
            continue
        print(toDate)
        #portfolio = pd.DataFrame(downloadlist)    
        trade_simulator(portfolio, toDate, cd=False, plot=False)
    
    

In [ ]:
if mode == "Algo":
    x = 'RELIANCE'
    gc.collect()
    days =2
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, cd=True, plot=True)+"%")

In [ ]:
if mode == "Algo":
    x = 'INFRATEL'
    gc.collect()
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, mode="simulator")+"%")